# Imports


In [ ]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

from tqdm.notebook import tqdm

#Dataset


Dataset from https://www.kaggle.com/puneet6060/intel-image-classification.

Guide to downloading Kaggle dataset directly into colab: https://www.kaggle.com/general/74235.

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d puneet6060/intel-image-classification 

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('intel-image-classification.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [ ]:
training_transforms = transforms.Compose([transforms.RandomRotation(30),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop(128),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406], 
                                                               [0.229, 0.224, 0.225])
                                          ])

validation_transforms = transforms.Compose([
                                            transforms.RandomResizedCrop(128),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])
                                            ])

In [ ]:
train_path = '/content/files/seg_train/seg_train'
valid_path = '/content/files/seg_train/seg_test'

In [ ]:
training_dataset = datasets.ImageFolder(train_path, transform = training_transforms )
validation_dataset = datasets.ImageFolder(train_path, transform = validation_transforms )

In [ ]:
training_loader = torch.utils.data.DataLoader(training_dataset, batch_size=32, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [ ]:
training_dataset.classes, validation_dataset.classes

num_classes = len(validation_dataset.classes)

There are 6 classes


#Create Model


Input image shape is 150*150 pixels with 3 RGB channels. 

###Notes:
All layers are found in nn:
* Fully Connected Layer: `Linear(num inputs, num outputs)`
* Max Pooling: `MaxPool2d(kernel size, stride)`
* Convolutional Layer: `Conv2d(input channels, output channels, kernel size, stride)`
* Define your layers in the `__init__` method of your model
* Flatten a PyTorch tensor using `.flatten()`

In [ ]:
class MyModel(nn.Module):
  def __init__(self, num_classes):
      super(MyModel,self).__init__()
      self.convo1 = nn.Conv2d(in_channels = 3,out_channels = 7,kernel_size= 5,stride = 1)
      self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
      self.convo2 = nn.Conv2d(in_channels = 7,out_channels = 14,kernel_size= 5,stride = 1)
      self.fc1 = nn.Linear(in_features=11774,out_features=64) 
      self.fc2 = nn.Linear(in_features=64, out_features=num_classes)
  
  def forward(self, x):
      x = self.convo1(x)
      x = torch.nn.functional.relu(x)
      x = self.pool(x)
      x = self.convo2(x)
      x = torch.nn.functional.relu(x)
      x = self.pool(x)
      x = torch.flatten(x, start_dim=1)
      x = self.fc1(x)
      x = torch.nn.functional.relu(x)

      x = self.fc2(x)
      return x

#Model Training

In [ ]:
# Function for the training 

def train(model, train_loader, loss_fn, optimizer, device):
    model.train() # puts the model in training mode
    running_loss = 0
    with tqdm(total=len(train_loader)) as pbar:
        for i, data in enumerate(train_loader, 0): # loops through training data
            inputs, labels = data # separate inputs and labels (outputs)
            inputs, labels = inputs.to(device), labels.to(device) # puts the data on the GPU

            # forward + backward + optimize                                          
            optimizer.zero_grad() # clear the gradients in model parameters
            outputs = model(inputs) # forward pass and get predictions
            loss = loss_fn(outputs, labels) # calculate loss
            loss.backward() # calculates gradient w.r.t to loss for all parameters in model that have requires_grad=True
            optimizer.step() # iterate over all parameters in the model with requires_grad=True and update their weights.

            running_loss += loss.item() # sum total loss in current epoch for print later

            pbar.update(1) #increment our progress bar

    return running_loss/len(train_loader) # returns the total training loss for the epoch

In [ ]:
# Function for the validation pass

def validation(model, val_loader, loss_fn, device):
    model.eval() # puts the model in validation mode
    running_loss = 0
    total = 0
    correct = 0
    
    with torch.no_grad(): # save memory by not saving gradients which we don't need 
        with tqdm(total=len(val_loader)) as pbar:
            for images, labels in iter(val_loader):
                images, labels = images.to(device), labels.to(device) # put the data on the GPU
                outputs = model(images) # passes image to the model, and gets a ouput which is the class probability prediction

                val_loss = loss_fn(outputs, labels) # calculates val_loss from model predictions and true labels
                running_loss += val_loss.item()
                _, predicted = torch.max(outputs, 1) # turns class probability predictions to class labels
                total += labels.size(0) # sums the number of predictions
                correct += (predicted == labels).sum().item() # sums the number of correct predictions
        
                pbar.update(1)

        return running_loss/len(val_loader), correct/total # return loss value, accuracy

#Model Instantiation


In [ ]:
model = MyModel(num_classes)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Determine whether a GPU is available
model.to(device) # send model to GPU

In [ ]:
loss_fn = nn.CrossEntropyLoss() # We use Cross Entropy Loss, as this is a classification task
optimizer = optim.Adam(model.parameters(), lr=0.0001) # If in doubt, we use Adam as our optimiser

#Training

In [ ]:
total_epoch = 20 # Define how many epochs of training we want

# keep track of things we'd like to plot later
training_losses = []
validation_losses = []
accuracies = []

for epoch in range(total_epoch): # loops through number of epochs
  train_loss = train(model, training_loader, loss_fn, optimizer, device)  # train the model for one epoch
  val_loss, accuracy = validation(model, validation_loader, loss_fn, device) # after training for one epoch, run the validation() function to see how the model is doing on the validation dataset
  
  # keep track of interesting stuff
  training_losses.append(train_loss)
  validation_losses.append(val_loss)
  accuracies.append(accuracy)
  
  print("Epoch: {}/{}, Training Loss: {}, Val Loss: {}, Val Accuracy: {}".format(epoch+1, total_epoch, train_loss, val_loss, accuracy))
  print('-' * 20)

print("Finished Training")

# Save the queen
torch.save(model.state_dict(), 'finished')